In [3]:
import os
import librosa
import matplotlib.pyplot as plt
import numpy as np
from pydub import AudioSegment
import h5py

In [15]:
def plot_mfcc_spectrogram(log_mel_spectrogram, sr):
    plt.figure(figsize=(10, 5))
    librosa.display.specshow(log_mel_spectrogram, sr=sr, hop_length=256, x_axis='time', y_axis='mel')
    print(log_mel_spectrogram.shape)
    plt.colorbar(format='%+2.0f dB')
    plt.title('Log Mel Spectrogram')
    plt.show()

def audio_to_spectrogram(src_dir, dest_dir):
    h5_arr = []
    j = 0
    with h5py.File(dest_dir, 'w') as hf:
        # Iterate over the FLAC files in the source directory
        for root, _, files in os.walk(src_dir, topdown=False):
            for file_name in files:
                if file_name.endswith('.flac') or file_name.endswith('.mp3'):
                    try:
                        # Load the FLAC file
                        input_file = os.path.join(root, file_name)
                        audio, sr = librosa.load(input_file, sr=22050)
                        sec = sr * 3

                        # Generate the spectrogram batches
                        for i in range(audio.shape[0]//(sr * 3)):
                            batch = audio[i*sec:(i+1)*sec]
                            
                            mel_spectrogram = librosa.feature.melspectrogram(y=batch, sr=sr, n_fft=4096, hop_length=512, n_mels=80)
                            
                            log_mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
                            h5_arr.append(log_mel_spectrogram)
                            
                            if j == 0:
                                print(batch.shape)
                                print(mel_spectrogram.shape)
                                print(log_mel_spectrogram.shape)
                                # mel_spectrogram_linear = librosa.db_to_power(log_mel_spectrogram, ref=np.max)
                                # stft =  np.abs(librosa.stft(mel_spectrogram_linear))
                                # spectral_centroids = librosa.feature.spectral_centroid(S=stft, sr=22050)
                                # print(np.mean(spectral_centroids))
                                j += 1
                            ## larger nfft better for frequency resolution
                            ## higher hop length for time resolution
                    except AttributeError as e:
                        print(f"Error processing file: {file_name} - {e}")
                        continue
        h5_arr = np.array(h5_arr)
        print(h5_arr.shape)
        hf.create_dataset("audio", data=h5_arr)

In [16]:
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
audio_to_spectrogram(os.path.join(parent_dir, 'iu'),'iu3.h5')
audio_to_spectrogram(os.path.join(parent_dir, 'bruno_mars'),'bruno3.h5')

(66150,)
(80, 130)
(80, 130)


KeyboardInterrupt: 

In [52]:
# h5 = h5py.File("iu.h5", 'r')
h5 = h5py.File("iu_spec.h5", 'r')
h5.close()

OSError: Unable to open file (file signature not found)